In [1]:
import numpy as np
import cupy as qp
import pylab as pl
import numba as nb
import timeit
xp = qp

In [2]:
NX = 30000
NY = 40000
nx = 1000
ny = 2000
steps = 30
overX = 10
overY = 10
np.ceil(NY/(ny-overY))

21.0

In [3]:
A = np.arange((NX*NY),dtype=np.float32).reshape((NX,NY)) 
AMAX = A.copy()
AMIN = A.copy() # ((NX,NY))

In [4]:
xp = qp
#@nb.njit()
#@xp.fuse() # this fails because fuse doesn't like indexing.
def diffit(A,B,DX):
    B[:,0:-1] = (A[:,1:]-A[:,0:-1])/DX
    B[:,-1]= B.dtype.type(0.0)  # I may not need this in the patchwise
    #print (B.dtype,B.shape)
    return 


In [5]:
#@nb.njit()
def blockcomp(B,DX,NX=NX,NY=NY,nx=nx,ny=ny,overX=overX,overY=overY):
    A = xp.empty((nx,ny),dtype=xp.float32)
    C = xp.empty((nx,ny),dtype=A.dtype)
    for i in range(0, NX, nx-overX):
        for j in range(0, NY,ny-overY):
            tmp = B[i:i+nx,j:j+ny]  # subscripts past ends will truncate!
            U = tmp.shape           # therefore we need to see what shape really is
            A[:U[0],:U[1]] = xp.asarray(tmp)
            AP = A[:U[0],:U[1]]
            D = C[:U[0],:U[1]]
            for s in range(steps):
                 diffit(AP,D,DX) 
                 AP,D = D,AP
            tmp[:,:] = qp.asnumpy(AP) # put it back in same home
blockcomp(A,2)               

In [6]:
#     ' int j = _ind.size() ; z = 0*x + 0*y[_ind.size() - i - 1]+i ;',
add_reverse = qp.ElementwiseKernel(
     'T x, raw T y, int64 w, T DX', 'T z',
     ' z = (y[i]-y[i+1])/DX ;',
     'add_reverse')
t0 = qp.arange(3,dtype=qp.float32)
t1 = qp.arange(12,dtype=qp.float32).reshape((3,4))
t2 = add_reverse(t0,t1,3,3)
print(qp.asnumpy(t2))
print(qp.asnumpy(t1))

#@qp.fuse()
def blockcomp2(B,BMAX,BMIN,DX,NX=NX,NY=NY,nx=nx,ny=ny,overX=overX,overY=overY):
    xp = qp
    DX = qp.float32(DX)
    A = xp.empty((nx,ny),dtype=xp.float32)
    
    C = xp.empty((nx,ny),dtype=A.dtype)
    AMAX = xp.empty((nx,ny),dtype=xp.float32)
    AMIN = xp.empty((nx,ny),dtype=xp.float32)
    for i in range(0, NX, nx-overX):
        for j in range(0, NY,ny-overY):
            tmp = B[i:i+nx,j:j+ny]  # subscripts past ends will truncate!
            U = tmp.shape           # therefore we need to see what shape really is
            A[:U[0],:U[1]] = xp.asarray(tmp)
            AP = A[:U[0],:U[1]]
            D = C[:U[0],:U[1]]
            
            AMAX[0:U[0],0:U[1]] =  AP[:,:]  # copy it
            AMIN[0:U[0],0:U[1]] =  AP[:,:]  # copy it
            
            
            for s in range(steps):
                D = add_reverse(AP,AP,U[1],DX)
                AP,D = D,AP
                AMAX[:,:] = xp.maximum(AP,AMAX)
                AMIN[:,:] = xp.minimum(AP,AMIN)
             tmp[:,:] = qp.asnumpy(AP) # put it back in same home
             BMAX[i:i+nx,j:j+ny] = np.maximum( qp.asnumpy(AMAX),BMAX[i:i+nx,j:j+ny])
             BMIN[i:i+nx,j:j+ny] = np.minimum( qp.asnumpy(AMIN, BMIN[i:i+nx,j:j+ny])
blockcomp2(A,2)       

[-0.33333334 -0.33333334 -0.33333334]
[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]]


In [7]:
%timeit blockcomp2(A,AMAX,AMIN,2)

3.37 s ± 3.47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
xp =qp
%timeit blockcomp(A,AMAX,AMIN,2)

6.06 s ± 4.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
xp = np
%timeit blockcomp(A,AMAX,AMIN,2)

51.6 s ± 38.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
add_reverse = qp.ElementwiseKernel(
     'T x, raw T y, int64 w', 'T z',
     ' z = y[i]-y[i+w] ;',
     'add_reverse')
t0 = qp.arange(3,dtype=qp.float32)
t1 = qp.arange(12,dtype=qp.float32).reshape((3,4))
t2 = add_reverse(t1,t1,4)
print(qp.asnumpy(t2))
print(qp.asnumpy(t1))

[[-4. -4. -4. -4.]
 [-4. -4. -4. -4.]
 [ 8.  9. 10. 11.]]
[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]]


In [11]:
t2 = add_reverse